In [7]:
import biosig as bs
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from scipy.signal import savgol_filter, find_peaks, correlate 
from numpy import argmax

In [145]:
FILENAME="/Users/lubimyj/Git/experiments/RAF/12_06_22_1k/Specimen7/WAVE0013.WFT"

HDR=bs.header(FILENAME)
data=bs.data(FILENAME)
sr = float(HDR.split('\n')[9][HDR.split('\n')[9].find(':')+2:-1])
time = np.arange(len(data))/sr
incid = [item[0]-data[0][0] for item in data]

In [146]:
FILENAME="/Users/lubimyj/Git/experiments/RAF/12_06_22_1k/Specimen7/WAVE0014.WFT"

HDR=bs.header(FILENAME)
data=bs.data(FILENAME)
sr = float(HDR.split('\n')[9][HDR.split('\n')[9].find(':')+2:-1])
time = np.arange(len(data))/sr
trans = [item[0]-data[0][0] for item in data]

In [147]:
fig1 = go.Figure(layout=dict(template = "none"))
fig1.add_trace(go.Scatter(x = time, y = incid, name = "Incident"))
fig1.add_trace(go.Scatter(x = time, y = trans, name = "Transmited"))
fig1.show()

In [148]:
sr

2000000.0

In [149]:
140e-6*sr

280.0

In [150]:
(2*0.66/5050)*sr

522.7722772277227

In [151]:
smooth_value = 57
sav_incid = savgol_filter(incid, smooth_value, 3)
sav_trans = savgol_filter(trans, smooth_value - 50, 3)

fig1 = go.Figure(layout=dict(template = "none"))
fig1.add_trace(go.Scatter(y = incid, name = "Incident"))
fig1.add_trace(go.Scatter(y = sav_incid, name = "smooth_Incident"))

fig1.add_trace(go.Scatter(y = trans, name = "Transmitted"))
fig1.add_trace(go.Scatter(y = sav_trans, name = "smooth_Transmitted"))

fig1.show()

In [152]:
int(0.66*2/5050*sr)

522

In [153]:
abs_incid = np.absolute(sav_incid)
abs_trans = np.absolute(sav_trans)

max_incid = max(sav_incid)

prominence = max_incid * 0.5
peaks_incid, _ = find_peaks(abs_incid, prominence=prominence)
while len(peaks_incid) < 4:
        prominence -= max_incid / 200
        peaks_incid, _ = find_peaks(abs_incid, prominence=prominence)

max_trans = max(sav_trans)
prominence = max_trans * 0.5

#   Find the peaks in the Transmitted signal and take only the time stamps
peaks_trans, _ = find_peaks(abs_trans, prominence=prominence)

while len(peaks_trans) < 4:
        prominence -= max_trans / 200
        peaks_trans, _ = find_peaks(abs_trans, prominence=prominence)
peaks_trans

array([ 757, 1156, 1279, 1674, 1797, 2191, 2317, 2708, 2830, 2832])

In [154]:
noize = np.random.random(len(incid))*2e-4

In [156]:
incid_before_idx = peaks_incid[0]
while incid[incid_before_idx]+noize[incid_before_idx] < 0:
    incid_before_idx -= 1

incid_after_idx = peaks_incid[0]
while incid[incid_after_idx]+noize[incid_after_idx] < 0:
    incid_after_idx += 1

    
spacing = 60
vcc_incid = incid[incid_before_idx - spacing: incid_after_idx + 1 * spacing]
time_incid = time[incid_before_idx - spacing: incid_after_idx + 1 * spacing]

signal_time = incid_after_idx - incid_before_idx

#   For reflected wave:
before_idx = peaks_incid[1]
while incid[before_idx] > 0:
    before_idx -= 1


before_idx = incid_before_idx + int(0.66*2/5050*sr)
#   Total cropping time
after_idx = before_idx + signal_time
reflected_idx = before_idx
vcc_reflected = incid[before_idx - spacing: after_idx + 1 * spacing]
time_reflected = time[before_idx - spacing: after_idx + 1 * spacing]

#   For transmitted wave:
before_idx = peaks_trans[0]

while trans[before_idx]+noize[before_idx] < 0:
    before_idx -= 1

#   Total cropping time
after_idx = before_idx + signal_time
vcc_trans = trans[before_idx - spacing: after_idx + 1 * spacing]
time_trans = time[before_idx - spacing: after_idx + 1 * spacing]


In [157]:
fig1 = go.Figure(layout=dict(template = "none"))
fig1.add_trace(go.Scatter(x = time_incid, y = vcc_incid, name = "Incident"))

fig1.add_trace(go.Scatter(x = time_trans, y = vcc_trans, name = "Transmitted"))
fig1.add_trace(go.Scatter(x = time_reflected, y = vcc_reflected, name = "Reflected"))

fig1.show()